## 학교 보건실 키 기록표

### 가상 데이터  

In [1]:
import numpy as np
import pandas as pd

# ---------------------------------------------------------
# 1) 재현 가능한 시드 고정
# ---------------------------------------------------------
np.random.seed(42)

# ---------------------------------------------------------
# 2) 시간축 생성 (1분 간격, 1일)
# ---------------------------------------------------------
start = "2026-02-02 09:00:00"
periods = 24 * 60  # 1440분
time_index = pd.date_range(start=start, periods=periods, freq="min")

# ---------------------------------------------------------
# 3) 설비 2대 만들기
# ---------------------------------------------------------
machine_ids = ["M01", "M02"]

rows = []
i = 0
while i < len(time_index):
    t = time_index[i]

    # M01 기본 센서(정상 범위)
    vib_m01 = 1.00 + 0.05 * np.random.randn()
    temp_m01 = 40.0 + 0.30 * np.random.randn()
    press_m01 = 10.0 + 0.10 * np.random.randn()

    # M02 기본 센서(조금 다른 분포)
    vib_m02 = 0.95 + 0.06 * np.random.randn()
    temp_m02 = 41.0 + 0.35 * np.random.randn()
    press_m02 = 10.2 + 0.12 * np.random.randn()

    rows.append([t, "M01", vib_m01, temp_m01, press_m01])
    rows.append([t, "M02", vib_m02, temp_m02, press_m02])

    i += 1

df_sensor = pd.DataFrame(
    rows,
    columns=["ts", "machine_id", "vibration", "temperature", "pressure"]
)

# ---------------------------------------------------------
# 4) 문자열 오염(실무 느낌): 라인명/공장명 컬럼 추가
# ---------------------------------------------------------
df_sensor["line"] = " Line-A "
df_sensor.loc[df_sensor["machine_id"] == "M02", "line"] = "line-a"
df_sensor["plant"] = "Seoul "
df_sensor.loc[df_sensor["machine_id"] == "M02", "plant"] = " seoul"

# ---------------------------------------------------------
# 5) 결측치 주입 (랜덤 10개)
# ---------------------------------------------------------
missing_idx = np.random.choice(df_sensor.index, size=10, replace=False)
df_sensor.loc[missing_idx, "temperature"] = np.nan

# ---------------------------------------------------------
# 6) 중복 행 주입 (랜덤 5개 행을 복제)
# ---------------------------------------------------------
dup_idx = np.random.choice(df_sensor.index, size=5, replace=False)
df_dup = df_sensor.loc[dup_idx].copy()
df_sensor = pd.concat([df_sensor, df_dup], axis=0, ignore_index=True)

# ---------------------------------------------------------
# 7) 이상치(스파이크) 주입: vibration/temp를 특정 시점에 크게 튀게 함
#    - 총 6개 스파이크(설비별 3개)
# ---------------------------------------------------------
spike_times_m01 = [
    pd.Timestamp("2026-02-02 11:00:00"),
    pd.Timestamp("2026-02-02 15:30:00"),
    pd.Timestamp("2026-02-02 20:10:00"),
]
spike_times_m02 = [
    pd.Timestamp("2026-02-02 10:20:00"),
    pd.Timestamp("2026-02-02 16:40:00"),
    pd.Timestamp("2026-02-02 22:50:00"),
]

j = 0
while j < len(spike_times_m01):
    t = spike_times_m01[j]
    mask = (df_sensor["ts"] == t) & (df_sensor["machine_id"] == "M01")
    df_sensor.loc[mask, "vibration"] = 3.2
    df_sensor.loc[mask, "temperature"] = 55.0
    j += 1

k = 0
while k < len(spike_times_m02):
    t = spike_times_m02[k]
    mask = (df_sensor["ts"] == t) & (df_sensor["machine_id"] == "M02")
    df_sensor.loc[mask, "vibration"] = 3.6
    df_sensor.loc[mask, "temperature"] = 58.0
    k += 1

# ---------------------------------------------------------
# 8) 설비 메타 테이블(df_machine) 생성 (merge 연습용)
# ---------------------------------------------------------
df_machine = pd.DataFrame({
    "machine_id": ["M01", "M02"],
    "model": ["X-100", "X-200"],
    "install_year": [2022, 2021],
    "owner_team": ["alpha", "beta"]
})

# ---------------------------------------------------------
# 9) 기본 확인 출력
# ---------------------------------------------------------
print("df_sensor shape:", df_sensor.shape)
print("df_machine shape:", df_machine.shape)
print(df_sensor.head(3))
print(df_machine)


df_sensor shape: (2885, 7)
df_machine shape: (2, 4)
                   ts machine_id  vibration  temperature   pressure      line  \
0 2026-02-02 09:00:00        M01   1.024836    39.958521  10.064769   Line-A    
1 2026-02-02 09:00:00        M02   1.041382    40.918046  10.171904    line-a   
2 2026-02-02 09:01:00        M01   1.078961    40.230230   9.953053   Line-A    

    plant  
0  Seoul   
1   seoul  
2  Seoul   
  machine_id  model  install_year owner_team
0        M01  X-100          2022      alpha
1        M02  X-200          2021       beta


In [2]:
df_sensor

,ts,machine_id,vibration,temperature,pressure,line,plant
0,2026-02-02 09:00:00,M01,1.024836,39.958521,10.064769,Line-A,Seoul
1,2026-02-02 09:00:00,M02,1.041382,40.918046,10.171904,line-a,seoul
2,2026-02-02 09:01:00,M01,1.078961,40.230230,9.953053,Line-A,Seoul
3,2026-02-02 09:01:00,M02,0.982554,40.837804,10.144112,line-a,seoul
4,2026-02-02 09:02:00,M01,1.012098,39.426016,9.827508,Line-A,Seoul
...,...,...,...,...,...,...,...
2880,2026-02-02 11:59:00,M02,0.924313,41.525266,10.302027,line-a,seoul
2881,2026-02-02 19:03:00,M01,1.059738,40.268394,9.862714,Line-A,Seoul
2882,2026-02-03 03:52:00,M01,1.044503,39.817822,9.979980,Line-A,Seoul
2883,2026-02-02 09:28:00,M01,0.987731,39.773879,9.911049,Line-A,Seoul


In [3]:
df_sensor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2885 entries, 0 to 2884
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ts           2885 non-null   datetime64[ns]
 1   machine_id   2885 non-null   object        
 2   vibration    2885 non-null   float64       
 3   temperature  2875 non-null   float64       
 4   pressure     2885 non-null   float64       
 5   line         2885 non-null   object        
 6   plant        2885 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 157.9+ KB


In [4]:
# df_sensor.value_counts(['machine_id','line','plant']) # 🐚이렇게 하면 세개의 묶음으로 보는거임 각각을 보려면 따로 파야함.
print(df_sensor['machine_id'].value_counts())
print(df_sensor['line'].value_counts())
print(df_sensor['plant'].value_counts())

machine_id
M01    1443
M02    1442
Name: count, dtype: int64
line
 Line-A     1443
line-a      1442
Name: count, dtype: int64
plant
Seoul     1443
 seoul    1442
Name: count, dtype: int64


In [5]:
df_machine

,machine_id,model,install_year,owner_team
0,M01,X-100,2022,alpha
1,M02,X-200,2021,beta


In [6]:
df_machine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   machine_id    2 non-null      object
 1   model         2 non-null      object
 2   install_year  2 non-null      int64 
 3   owner_team    2 non-null      object
dtypes: int64(1), object(3)
memory usage: 196.0+ bytes


### 미션 1. 문자열 표준화
- line과 plant를 표준화해서 새 컬럼 line_std, plant_std 만들기
- 공백 제거 + 케이스 통일

In [7]:
# ⭐️ df_sensor['line_std'].replace({"a":"A"},{"l":"L"})
# 🐚 replace 는 정확히 일치해야만 바꾸는거임. 부분적으로 일치해도 안바뀐다. 딕셔너리 형식은 .str를 안써도 되고 부분적으로 바꾸고싶으면 .str.replace(a,b)를 해야한다. 
# 🐚 또한 매단계마다 str를 붙이고 해당 함수를 작성해야함. 

In [8]:
df_sensor['line_std'] = df_sensor['line'].str.strip().replace({"line-a":"Line-A"}) # 딕셔너리 이용방법 
# df_sensor['line_std'] = df_sensor['line'].str.strip().str.replace("a","A").str.replace("l","L") # str 부분 변환 방법 주의 🐚 모두 바뀔수있음. 
df_sensor['line_std']

df_sensor['plant_std'] = df_sensor['plant'].str.strip().str.replace("s","S")
df_sensor['plant_std']


0       Seoul
1       Seoul
2       Seoul
3       Seoul
4       Seoul
        ...  
2880    Seoul
2881    Seoul
2882    Seoul
2883    Seoul
2884    Seoul
Name: plant_std, Length: 2885, dtype: object

In [9]:
print(df_sensor['plant_std'].value_counts())
print(df_sensor['line_std'].value_counts())

plant_std
Seoul    2885
Name: count, dtype: int64
line_std
Line-A    2885
Name: count, dtype: int64


### 미션 2: 중복제거 
- ["ts", "machine_id"] 기준으로 중복 제거하고 행 수가 줄어드는지 확인

In [10]:
# 🐚 중복확인 자체가  duplicates / 중복 제거가 drop_duplicates 
# --> 잘못된 중복 확인 방법: print(f"중복 제거전 {df_sensor[["ts","machine_id"]].value_counts()} 와 개수 {len(df_sensor[["ts","machine_id"]])}\n") 
# --> 잘못된 제거 방법:# print(f"중복 제거후 {df_sensor[["ts","machine_id"]].value_counts()} 와 개수 {len(df_sensor[["ts","machine_id"]])}")

print(df_sensor.duplicated(subset=["ts","machine_id"]).sum()) # 5개의 중복 확인
df_sensor = df_sensor.drop_duplicates(subset=["ts","machine_id"],keep='first')
print(df_sensor.duplicated(subset=["ts","machine_id"]).sum()) # 0개의 중복 확인

5
0


### 미션 3: 이상치 탐지
- 설비별로 vibration에 대해
- 3-sigma 기준 이상치 플래그 is_anom_3sigma
- IQR 기준 이상치 플래그 is_anom_iqr

In [11]:
# 3-sigma 
vibration_std = df_sensor['vibration'].std()
vibration_mean = df_sensor['vibration'].mean()

df_sensor['3sigma_upper'] = vibration_mean + (3 * vibration_std)
df_sensor['3sigma_lower'] = vibration_mean - (3 * vibration_std)

# 해당하는것만 남김. 즉 정상만 남긴거.
df_sensor['is_anom_3sigma_mask'] = (df_sensor['vibration'] >= df_sensor['3sigma_lower']) & (df_sensor['vibration'] <= df_sensor['3sigma_upper']) 

/var/folders/bf/_c9d0b9571x410m1fmyxr4yh0000gn/T/ipykernel_5074/1671019210.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sensor['3sigma_upper'] = vibration_mean + (3 * vibration_std)
/var/folders/bf/_c9d0b9571x410m1fmyxr4yh0000gn/T/ipykernel_5074/1671019210.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sensor['3sigma_lower'] = vibration_mean - (3 * vibration_std)
/var/folders/bf/_c9d0b9571x410m1fmyxr4yh0000gn/T/ipykernel_5074/1671019210.py:9: SettingWithCopyWarning: 
A value is trying to b

In [12]:
# IQR
q1 = df_sensor['vibration'].quantile(0.25)
q3 = df_sensor['vibration'].quantile(0.75)

IQR = q3- q1 
# IQR 해당하는 범위만 남김.
df_sensor['IQR'] = ((q1 - IQR * 1.5) <= df_sensor['vibration']) & (df_sensor['vibration'] <= (q3 + IQR * 1.5))

/var/folders/bf/_c9d0b9571x410m1fmyxr4yh0000gn/T/ipykernel_5074/782885865.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sensor['IQR'] = ((q1 - IQR * 1.5) <= df_sensor['vibration']) & (df_sensor['vibration'] <= (q3 + IQR * 1.5))


In [13]:
df_3sigma = df_sensor[df_sensor['is_anom_3sigma_mask'] == False]
df_3sigma

,ts,machine_id,vibration,temperature,pressure,line,plant,line_std,plant_std,3sigma_upper,3sigma_lower,is_anom_3sigma_mask,IQR
161,2026-02-02 10:20:00,M02,3.6,58.0,10.029330,line-a,seoul,Line-A,Seoul,1.36022,0.598134,False,False
240,2026-02-02 11:00:00,M01,3.2,55.0,9.865181,Line-A,Seoul,Line-A,Seoul,1.36022,0.598134,False,False
780,2026-02-02 15:30:00,M01,3.2,55.0,10.049202,Line-A,Seoul,Line-A,Seoul,1.36022,0.598134,False,False
921,2026-02-02 16:40:00,M02,3.6,58.0,10.298441,line-a,seoul,Line-A,Seoul,1.36022,0.598134,False,False
1340,2026-02-02 20:10:00,M01,3.2,55.0,9.814816,Line-A,Seoul,Line-A,Seoul,1.36022,0.598134,False,False
1661,2026-02-02 22:50:00,M02,3.6,58.0,10.396296,line-a,seoul,Line-A,Seoul,1.36022,0.598134,False,False


In [14]:
df_3sigma.shape

(6, 13)

In [15]:
df_IQR = df_sensor[df_sensor['IQR'] == False]
df_IQR

,ts,machine_id,vibration,temperature,pressure,line,plant,line_std,plant_std,3sigma_upper,3sigma_lower,is_anom_3sigma_mask,IQR
161,2026-02-02 10:20:00,M02,3.600000,58.000000,10.029330,line-a,seoul,Line-A,Seoul,1.36022,0.598134,False,False
240,2026-02-02 11:00:00,M01,3.200000,55.000000,9.865181,Line-A,Seoul,Line-A,Seoul,1.36022,0.598134,False,False
367,2026-02-02 12:03:00,M02,0.776225,41.730931,10.183249,line-a,seoul,Line-A,Seoul,1.36022,0.598134,True,False
513,2026-02-02 13:16:00,M02,0.777664,40.990370,10.412670,line-a,seoul,Line-A,Seoul,1.36022,0.598134,True,False
657,2026-02-02 14:28:00,M02,0.773577,41.229565,10.223368,line-a,seoul,Line-A,Seoul,1.36022,0.598134,True,False
780,2026-02-02 15:30:00,M01,3.200000,55.000000,10.049202,Line-A,Seoul,Line-A,Seoul,1.36022,0.598134,False,False
921,2026-02-02 16:40:00,M02,3.600000,58.000000,10.298441,line-a,seoul,Line-A,Seoul,1.36022,0.598134,False,False
965,2026-02-02 17:02:00,M02,1.185574,40.270560,10.406964,line-a,seoul,Line-A,Seoul,1.36022,0.598134,True,False
1097,2026-02-02 18:08:00,M02,0.800731,41.446938,10.240563,line-a,seoul,Line-A,Seoul,1.36022,0.598134,True,False
1135,2026-02-02 18:27:00,M02,0.798065,41.238724,10.215141,line-a,seoul,Line-A,Seoul,1.36022,0.598134,True,False


In [16]:
df_IQR.shape

(31, 13)

### 미션 4: 리포트
- 설비별 이상치 개수(3-sigma)를 집계해서 표로 출력 
- 개수, 비율, 시간대 나오기
- (보너스) df_machine과 merge해서 모델명까지 붙여 출력

In [17]:
df_machine

,machine_id,model,install_year,owner_team
0,M01,X-100,2022,alpha
1,M02,X-200,2021,beta


In [25]:
# 🐚 개수 + 비율 + (필요하면) 시간대를 어찌붙이지 ? 집계컬럼에서 더붙이거 .agg해서 집계함수 더 써도 뭔가 갑자기 비율인데 count해버리면 안되니까 이상할거같은데 [1,2,3].agg[1,2,3]하면 11 22 33 이렇게 되는건가?
df_report = df_sensor.groupby('machine_id',as_index=False).agg(anom_count = ("is_anom_3sigma_mask",'sum'),total = ("machine_id","count")) # 🐚 계산에서 번거로워지니까 이럴떄 불량자체를 세는것이 향후에 좋다 .즉 범위 밖은 False이 아니라 범위 밖일때True 가 낫다는뜻.
df_report['비율'] = df_report['anom_count'] / df_report['total']
df_report_plus = pd.merge(df_report,df_machine,how='inner',on='machine_id')
df_report_plus

,machine_id,anom_count,total,비율,model,install_year,owner_team
0,M01,1437,1440,0.997917,X-100,2022,alpha
1,M02,1437,1440,0.997917,X-200,2021,beta


In [ ]:
# 시간대 나오게 하는법 
df_sensor['ts'].dtype
df_sensor['ts'] = pd.to_datetime(df_sensor['ts'])

df_tmp = df_sensor.copy()
df_tmp['hour'] = df_tmp['ts'].dt.hour #🐚 df 는 str와 같은 datetime 도구상자임. 


report_hour = (
    df_tmp.groupby(["machine_id", "hour"], as_index=False)
          .agg(
              anom_cnt=("is_anom_3sigma_mask", "sum"),
              total=("is_anom_3sigma_mask", "size")
          )
)

report_hour["anom_rate"] = report_hour["anom_cnt"] / report_hour["total"]
report_hour = report_hour.sort_values(["machine_id", "hour"])
print(report_hour.head(10))

  machine_id  hour  anom_cnt  total  anom_rate
0        M01     0        60     60        1.0
1        M01     1        60     60        1.0
2        M01     2        60     60        1.0
3        M01     3        60     60        1.0
4        M01     4        60     60        1.0
5        M01     5        60     60        1.0
6        M01     6        60     60        1.0
7        M01     7        60     60        1.0
8        M01     8        60     60        1.0
9        M01     9        60     60        1.0


/var/folders/bf/_c9d0b9571x410m1fmyxr4yh0000gn/T/ipykernel_5074/1625748269.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sensor['ts'] = pd.to_datetime(df_sensor['ts'])
